In [1]:
import pandas as pd
import numpy as np
import re
import json
import nltk
from word2number import w2n
from nltk.stem.snowball import SnowballStemmer
import random

In [4]:
with open("../data/math23k_dolphin.json",'r') as f:
    dff = json.load(f)
dff = pd.read_json(dff)
dff = dff.T

In [5]:
def postfix_equation(equ_list):
    stack = []
    post_equ = []
    op_list = ['+', '-', '*', '/', '^']
    priori = {'^': 3, '*': 2, '/': 2, '+': 1, '-': 1}
    for elem in equ_list:
        if elem == '(':
            stack.append('(')
        elif elem == ')':
            while 1:
                op = stack.pop()
                if op == '(':
                    break
                else:
                    post_equ.append(op)
        elif elem in op_list:
            while 1:
                if not stack:
                    break
                elif stack[-1] == '(':
                    break
                elif priori[elem] > priori[stack[-1]]:
                    break
                else:
                    op = stack.pop()
                    post_equ.append(op)
            stack.append(elem)
        else:
            post_equ.append(elem)
    while stack:
        post_equ.append(stack.pop())
    return post_equ

def post_solver(post_equ):
    try:
        stack = [] 
        op_list = ['+', '-', '/', '*', '^']
        for elem in post_equ:
            if elem not in op_list:
                op_v = elem
                if '%' in op_v:
                    op_v = float(op_v[:-1])/100.0
                stack.append(str(op_v))
            elif elem in op_list:
                op_v_1 = stack.pop()
                op_v_1 = float(op_v_1)
                op_v_2 = stack.pop()
                op_v_2 = float(op_v_2)
                if elem == '+':
                    stack.append(str(op_v_2+op_v_1))
                elif elem == '-':
                    stack.append(str(op_v_2-op_v_1))
                elif elem == '*':
                    stack.append(str(op_v_2*op_v_1))
                elif elem == '/':
                    if op_v_1 == 0:
                        return nan
                    stack.append(str(op_v_2/op_v_1))
                else:
                    stack.append(str(op_v_2**op_v_1))
        return stack.pop()
    except:
        return None

def getBaseline(lst,flag=False):
    if(flag):
        lst = list(np.random.permutation(lst))
    ops = ['-','+','*','/']
    ops_nodiv = ['-','+','*']
    eq_list = [str(lst[0])]
    for i in range(1,len(lst)):
        if(lst[i][0]=='0'):
            op = random.choice(ops_nodiv)
        else:
            op = random.choice(ops)
        eq_list.append(op)
        eq_list.append(str(lst[i]))
    peq = postfix_equation(eq_list)
    return post_solver(peq) 

In [7]:
best = 0.0
for i in range(10):
    dff["baseline"] = dff["num_list"].apply(lambda x : getBaseline(x,True))
    a_ = dff["ans"]
    b_ = dff["baseline"]
    a,b = [],[]
    for i in range(len(a_)):
        if ';' in a_[i] or 'e' in a_[i]:
            continue
        if b_[i] is None:
            continue
        a.append(a_[i])
        b.append(b_[i])
    a = [int(float(x)) for x in a]
    b = [int(float(x)) for x in b]
    acc = sum(1 for x,y in zip(a,b) if x == y) / len(a)
    if(acc>best):
        best=acc
best
    

0.06972352085203876

In [8]:
best = 0.0
for i in range(10):
    dff["baseline"] = dff["num_list"].apply(lambda x : getBaseline(x,False))
    a_ = dff["ans"]
    b_ = dff["baseline"]
    a,b = [],[]
    for i in range(len(a_)):
        if ';' in a_[i] or 'e' in a_[i]:
            continue
        if b_[i] is None:
            continue
        a.append(a_[i])
        b.append(b_[i])
    a = [int(float(x)) for x in a]
    b = [int(float(x)) for x in b]
    acc = sum(1 for x,y in zip(a,b) if x == y) / len(a)
    if(acc>best):
        best=acc
best

0.07381033468528438